In [ ]:
from algorithm_module import start_amc, init_classification, combine_accuracy_graphs

samples, labels, snr, feature_list = start_amc()

In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import os
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.metrics import confusion_matrix
import seaborn as sns
import h5py
import pandas as pd


In [ ]:

# split data for training using only high SNR
def threshold_split(samples,labels,snr):
    # get the train set to be only positive SNR
    mask = snr>0
    samples_mask = samples[mask]
    labels_mask = labels[mask]
    snr_mask = snr[mask]
    samples_not = samples[~mask]
    labels_not = labels[~mask]
    x_train, x_test, y_train, y_test, train_indices, test_indices = train_test_split(
        samples_mask,
        labels_mask,
        range(len(samples_mask)),
        test_size=0.3,
        random_state=42,
        stratify=labels_mask,
        )
    x_test=np.concatenate((x_test,samples_not))
    y_test=np.concatenate((y_test,labels_not))
    snr_test = np.concatenate((snr_mask[test_indices],snr[~mask]))
    return x_train, x_test, y_train, y_test,snr_test

def get_accuracy(samples, labels, snr, classifier, feat_pick):
    # make a confusion matrix and give the detection probability
    unique_snr = np.unique(snr)
    unique_labels = np.unique(labels)
    accuracy_list = []
    for snr_val in unique_snr:
        mask = snr == snr_val
        x_snr = samples[mask]
        y_snr = labels[mask]
        final_labels = classifier.predict(x_snr,feat_pick)
        accuracy_snr = accuracy_score(y_snr, final_labels)
        accuracy_list.append(accuracy_snr)
        if snr_val in [6,10,20]:
            cm = confusion_matrix(y_snr, final_labels, labels=unique_labels)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_labels, yticklabels=unique_labels)
            plt.xlabel("Predicted Labels")
            plt.ylabel("True Labels")
            plt.title(f"Confusion Matrix for SNR={snr_val}dB, Acc={accuracy_snr}%")
            file_path = f"confusion_matrix_SNR={snr_val}.png"
            plt.savefig(file_path)
            plt.clf()
            plt.close()
    accuracy_steps  = {"snr": unique_snr, "accuracy": accuracy_list, "name": 'Steps Classification'}
    return accuracy_steps

def label_detection(x_test, y_test, snr_test, name, classifier,feat_pick):
    # shows the detection probability per label
    group1 = ['AM-DSB-SC','AM-SSB-SC','FM','GMSK','BPSK']
    group2 = ['OOK','4ASK','OQPSK','QPSK','8PSK','16QAM']
    groups = [group1, group2]   
    # classify per label
    unique_snr = np.unique(snr_test)
    num_plots = len(groups)  # Number of plots needed
    fig, axs = plt.subplots(nrows=num_plots, ncols=1, figsize=(10, 6 * num_plots))
    for i, label_group in enumerate(groups):
        axs[i].set_title(f"Accuracy vs SNR for Label Group {i+1}")
        for label in label_group:
            mask = y_test == label
            x_label = x_test[mask]
            y_label = y_test[mask]
            snr_label = snr_test[mask]
            accuracy_list = []
            for snr_val in unique_snr:
                mask_snr = snr_label == snr_val
                x_snr = x_label[mask_snr]
                y_snr = y_label[mask_snr]
                y_pred_snr = classifier.predict(x_snr,feat_pick)
                accuracy_snr = accuracy_score(y_snr, y_pred_snr)
                accuracy_list.append(accuracy_snr)
            axs[i].plot(unique_snr, accuracy_list, label=label)
        axs[i].set_xlabel("SNR")
        axs[i].set_ylabel("Accuracy")
        axs[i].legend(loc='lower right')
        axs[i].yaxis.set_major_locator(MultipleLocator(0.1))
        axs[i].xaxis.set_major_locator(MultipleLocator(2))
        axs[i].grid(which='major')
    plt.tight_layout()
    file_path =  f"{name}.png"
    plt.savefig(file_path)
    print("saved plots")
    plt.clf()
    plt.close()

In [ ]:
class SequentialClassifier:
    # the main classifier in my work
    # constructed from steps of regular classifiers in a row
    def __init__(self,groups):
        self.classifiers = []
        self.groups = groups
        
    def add_classifier(self, classifier):
        self.classifiers.append(classifier)
    
    def fit(self, samples, labels,feat_pick):     
        dataset = samples.copy()
        labels_group = labels.copy()
        for idx,group in enumerate(self.groups):
                mask = np.isin(labels_group,group)
                labels_fit = labels_group.copy()
                labels_fit[~mask]='others'
                data = dataset[:,feat_pick[idx]]
                self.classifiers[idx].fit(data,labels_fit)
                labels_group = labels_group[~mask]
                dataset = dataset[~mask]

    def predict(self,samples,feat_pick):
        dataset = samples.copy()
        labels_predict = np.full(len(samples),'others', dtype='<U9')
        for idx, group in enumerate(self.groups):
            data = dataset[:,feat_pick[idx]]
            step_predict = self.classifiers[idx].predict(data)
            mask = (labels_predict == 'others') & np.isin(step_predict, group)
            labels_predict[mask] = step_predict[mask]
        return labels_predict


In [ ]:


from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

class DoubleClassifier:        
    # The sub-classifier that helps. contain 2 steps
    # first step is seperating a pair of labels from the rest (based on similar groups)
    # second step is a major classifier between the 2 groups
    
    def __init__(self,depth,label_pair):
        self.classifiers = []
        self.classifiers.append(DecisionTreeClassifier(max_depth=depth))
        self.classifiers.append(SVC(kernel='linear', C=0.7, max_iter=1000))
        self.groups = label_pair
        self.encoder = LabelEncoder()
        
    def fit(self, samples, labels, feat_pick):     
            dataset = samples.copy()
            labels_group = labels.copy()
            mask = np.isin(labels_group,self.groups)
            labels_group[~mask]='others'
            labels_group[mask]='pair'
            data = dataset[:,feat_pick]
            self.classifiers[0].fit(data,labels_group)
            label_svm = self.encoder.fit_transform(labels[mask])
            # label_svm = labels[mask]
            data_svm = data[mask]
            self.classifiers[1].fit(data_svm,label_svm)
            print('SVM fit done')

    def predict(self,samples,feat_pick):
        dataset = samples.copy()
        data = dataset[:,feat_pick]
        step_predict = self.classifiers[0].predict(data)
        mask = step_predict == 'pair'
        step2_predict_encoded = self.classifiers[1].predict(data)
        step2_predict = self.encoder.inverse_transform(step2_predict_encoded)
    
        step_predict[mask] = step2_predict[mask]
        
        return step_predict
        


In [ ]:
# quick check for the double classifier

x_train, x_test, y_train, y_test, snr_test = threshold_split(samples,labels,snr)

a = DoubleClassifier(2,['AM-DSB-SC','BPSK'])
a.fit(x_train,y_train,[2,4,9,10])
y_predict = a.predict(x_test,[2,4,9,10])

mask = np.isin(y_test,['AM-DSB-SC','BPSK'])
y_pair = y_test[mask]
y_pair_predict = y_predict[mask]
snr_pair = snr_test[mask]
unique_labels = ['AM-DSB-SC','BPSK','others']

for snr_val in [2,6,10,20,30]:
    mask_snr = snr_pair == snr_val
    y_pair_snr = y_pair[mask_snr]
    y_pair_predict_snr = y_pair_predict[mask_snr]
    accuracy_snr = accuracy_score(y_pair_snr, y_pair_predict_snr)
    cm = confusion_matrix(y_pair_snr, y_pair_predict_snr, labels=unique_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_labels, yticklabels=unique_labels)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix SNR={snr_val}dB, Acc={accuracy_snr}%")
    file_path = f"confusion_matrix_SNR={snr_val}.png"
    plt.savefig(file_path)
    plt.clf()
    plt.close()


In [ ]:
# results for the main classifier

x_train, x_test, y_train, y_test, snr_test = threshold_split(samples,labels,snr)
Groups = [
        ['AM-SSB-SC'], # group 0
        ['AM-DSB-SC','BPSK'],# group 1
        ['4ASK','OOK'],# group 2
        ['GMSK','FM'],# group 3
        ['16QAM','8PSK','QPSK','OQPSK']] # group 4

# feat_pick = [[7,6,5,4],[10,9,2,0],[1,0,6,2],[3,0,7,4],[13,11,8,10],[2,4,7,5]]
feat_pick = [[7,6,5,4],[2,4,9,10],[1,0,6,2],[3,0,7,4],[13,11,8,10]]
depth = [1,2,1,1,1]
full_classifier = SequentialClassifier(Groups)

for ii in range(len(Groups)):
        if ii in [1]:
                print(1)
                classifier = SequentialClassifier(depth[ii],['AM-DSB-SC','BPSK'])
        else: 
                classifier = DecisionTreeClassifier(max_depth=depth[ii])
        full_classifier.add_classifier(classifier)

full_classifier.fit(x_train,y_train,feat_pick)
labels_predict = classifier.predict(x_test,y_test,feat_pick)
accuracy_step_3 = get_accuracy(x_test, y_test, snr_test, full_classifier, feat_pick)
label_detection(x_test, y_test, snr_test, 'Step Classification depth = 5', full_classifier, feat_pick)

# classifier4 = SequentialClassifier(4)
# classifier4.fit_model(x_train,y_train,feat_pick)
# # labels_predict = classifier.classify(x_test,y_test,feat_pick)
# accuracy_step_4 = get_accuracy(samples, labels, snr, classifier4, feat_pick)
# label_detection(x_test, y_test, snr_test, 'Step Classification depth = 4', classifier4,feat_pick)

# accuracy_data3,classifier = init_classification(samples, labels, snr, "Decision Tree depth 15", 0, 15)

# combine_accuracy_graphs(accuracy_data3,accuracy_step_3)

In [ ]:
# #DNN classifier
# import tensorflow as tf
# from tensorflow.keras import layers, models

# model = models.Sequential()
# model.add(layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(np.unique(y_train).shape[0], activation='softmax'))
# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=1, batch_size=32)
